In [2]:
import cv2
from pyzbar.pyzbar import decode
from datetime import datetime
import openpyxl
import os

# === File paths ===
box_file = "scanned_barcodes.xlsx"
product_file = "products.xlsx"

def init_excel(file_path):
    if not os.path.exists(file_path):
        wb = openpyxl.Workbook()
        ws = wb.active
        ws.append(["ID", "Barcode", "Timestamp"])
        wb.save(file_path)
    wb = openpyxl.load_workbook(file_path)
    ws = wb.active
    existing = {}
    next_id = 1
    for row in ws.iter_rows(min_row=2, values_only=True):
        id_, barcode, _ = row
        existing[barcode] = id_
        if id_ >= next_id:
            next_id = id_ + 1
    return wb, ws, existing, next_id

# === Initialize Excel files ===
box_wb, box_ws, box_scanned, box_next_id = init_excel(box_file)
prod_wb, prod_ws, prod_scanned, prod_next_id = init_excel(product_file)

# === Webcam Start ===
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")

print("Scanning... '3S' barcodes go to boxes file, others to products. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    barcodes = decode(frame)
    for barcode in barcodes:
        x, y, w, h = barcode.rect
        barcode_data = barcode.data.decode('utf-8')
        barcode_type = barcode.type
        display_text = f'{barcode_type}: {barcode_data}'

        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, display_text, (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        if barcode_data.startswith("3S"):
            if barcode_data not in box_scanned:
                box_ws.append([box_next_id, barcode_data, timestamp])
                box_wb.save(box_file)
                print(f"📦 Box saved: ID={box_next_id}, {barcode_data}")
                box_scanned[barcode_data] = box_next_id
                box_next_id += 1
        else:
            if barcode_data not in prod_scanned:
                prod_ws.append([prod_next_id, barcode_data, timestamp])
                prod_wb.save(product_file)
                print(f"🧃 Product saved: ID={prod_next_id}, {barcode_data}")
                prod_scanned[barcode_data] = prod_next_id
                prod_next_id += 1

    cv2.imshow("Barcode Scanner", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Scanning... '3S' barcodes go to boxes file, others to products. Press 'q' to quit.
🧃 Product saved: ID=10, A4846885A
🧃 Product saved: ID=11, 48836138


KeyboardInterrupt: 